<a href="https://colab.research.google.com/github/boyoung21/ESAA/blob/main/11_21_%EC%84%B8%EC%85%98_%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CIFAR-10 데이터셋을 이용한 CNN 모델

## Step 1. 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [2]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Step 2. CIFAR-10 데이터셋 다운로드

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화 추가
])

# 데이터셋 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

100%|██████████| 170M/170M [00:03<00:00, 44.0MB/s]


##Step 3. DataLoader 설정

In [4]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Step 4. CNN 모델 정의

In [8]:
# CNN 모델 정의

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()

        # 첫 번째 합성곱 층
        self.conv1 = nn.Conv2d(in_channels=3, out_channels = 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화 채널 수 입력

        # 두 번째 합성곱 층
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)

        # 세 번째 합성곱 층
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)

        # 풀링 층 (MaxPooling)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc1 = nn.Linear(128*2*2, 256)  # CIFAR-10은 32x32 크기
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # 활성화 함수 적용
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)  # Flatten

        x = F.relu(self.fc1(x))  # Fully Connected 활성화 함수 추가
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Step 5. 모델 학습 설정

In [9]:
# 모델을 초기화하세요.
model = CIFAR10_CNN().to(device)

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Step 6. 모델 학습 루프

In [10]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 2로 설정합니다.
# 런타임 약 3분
num_epochs = 2
for epoch in range(num_epochs):
   for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값을 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()

   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/2], Loss: 1.2474
Epoch [2/2], Loss: 0.7752


## Step 7. 테스트 정확도 평가

In [11]:
# 모델의 성능 평가

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 67.50%


# 2. MNIST 데이터셋 분류 모델




## Step 1. 필요한 라이브러리 임포트

In [12]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [13]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Step 2. MNIST 데이터셋 다운로드

In [14]:
# MNIST 데이터셋을 다운로드하고, 텐서로 변환할 수 있도록 필요한 전처리 함수를 추가하세요.
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 502kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.69MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.4MB/s]


##Step 3. DataLoader 설정

In [15]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Step 4. CNN 모델 정의

In [16]:
# CNN 모델 정의
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size= 3, padding = 1),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 256),
    nn.ReLU(),
    nn.Dropout(0.25),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

## Step 5. 모델 학습 설정

In [17]:
# 모델을 초기화하세요.

cnn = model.to(device)
# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)


## Step 6. 모델 학습 루프

In [18]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 2로 설정합니다.
# 런타임 약 3분

for epoch in range(2):
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = cnn(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Epoch [{epoch+1}/{2}], Loss: {loss.item():.4f}")

Epoch [2/2], Loss: 0.0161


## Step 7. 테스트 정확도 평가

In [21]:
# 모델의 성능 평가
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 98.73%
